# Stoichiometric reactions

**The biosteam.reaction package contains array based obejcts that can model stoichiometric reactions given a conversion.**

### Single reaction

Create a Reaction object based on the transesterification reaction:

|Reaction|Reactant|% Converted|
|---|---|---|
|Lipid + 3Methanol -> 3Biodiesel + Glycerol|Lipid|90|


In [1]:
from biosteam import Stream, reaction
from lipidcane.species import biodiesel_species
rxn = reaction
Stream.species = biodiesel_species

transesterification = rxn.Reaction('Lipid + 3Methanol -> 3Biodiesel + Glycerol',
                                   reactant='Lipid',  X=0.9)
transesterification

Reaction:
 stoichiometry                                 reactant    X[%]
 Lipid + 3 Methanol -> Glycerol + 3 Biodiesel  Lipid      90.00


In [2]:
transesterification.species

<WorkingSpecies: Lipid, Methanol, Glycerol, Biodiesel, Water, NaOH, HCl, NaOCH3>

In [3]:
transesterification.stoichiometry

array([-1., -3.,  1.,  3.,  0.,  0.,  0.,  0.])

In [4]:
transesterification.reactant

'Lipid'

In [5]:
transesterification.X

0.9

When a Reaction object is called with array (with each entry corresponding to species), it updates the array to reflect the change in material due to the reaction:

In [6]:
feed = Stream(Lipid=100, Methanol=600)
print('BEFORE REACTION')
feed.show()

# React feed molar flow rate
transesterification(feed.mol)

print('AFTER REACTION')
feed.show()

BEFORE REACTION
Stream: d1
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): Lipid     100
                 Methanol  600
AFTER REACTION
Stream: d1
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): Lipid      10
                 Methanol   330
                 Glycerol   90
                 Biodiesel  270


### Parallel reactions

Model the pretreatment hydrolysis reactions and assumed conversions from Humbird et. al. as shown in the follwing table [[1]](#References):

|Reaction|Reactant|% Converted|
|---|---|---|
|(Glucan)n + n H2O→ n Glucose|Glucan|9.9|
|(Glucan)n + n H2O → n Glucose Oligomer|Glucan|0.3|
|(Glucan)n → n HMF + 2n H2O|Glucan|0.3|
|Sucrose → HMF + Glucose + 2 H2O|Sucrose|100.0|
|(Xylan)n + n H2O→ n Xylose|Xylan|90.0|
|(Xylan)n + m H2O → m Xylose Oligomer|Xylan|2.4|
|(Xylan)n → n Furfural + 2n H2O|Xylan|5.0|
|Acetate → Acetic Acid|Acetate|100.0|
|(Lignin)n → n Soluble Lignin|Lignin|5.0|


Create a ParallelReaction from Reaction objects:

In [7]:
from cornstover.species import species 

# Set Stream.species as defined in [1-4]
Stream.species = species

# Create reactions
pretreatment_rxn = rxn.ParallelReaction([
    #            Reaction definition                 Reactant    Conversion
    rxn.Reaction('Glucan + H2O -> Glucose',          'Glucan',   0.0990),
    rxn.Reaction('Glucan + H2O -> GlucoseOligomer',  'Glucan',   0.0030),
    rxn.Reaction('Glucan -> HMF + 2 H2O',            'Glucan',   0.0030),
    rxn.Reaction('Sucrose -> HMF + Glucose + 2H2O',  'Sucrose',  0.0030),
    rxn.Reaction('Xylan + H2O -> Xylose',            'Xylan',    0.9000),
    rxn.Reaction('Xylan + H2O -> XyloseOligomer',    'Xylan',    0.0024),
    rxn.Reaction('Xylan -> Furfural + 2 H2O',        'Xylan',    0.0050),
    rxn.Reaction('Acetate -> AceticAcid',            'Acetate',  1.0000),
    rxn.Reaction('Lignin -> SolubleLignin',          'Lignin',   0.0050)])

pretreatment_rxn

ParallelReaction:
 stoichiometry                     reactant    X[%]
 H2O + Glucan -> Glucose           Glucan      9.90
 H2O + Glucan -> GlucoseOligomer   Glucan      0.30
 Glucan -> 2 H2O + HMF             Glucan      0.30
 Sucrose -> 2 H2O + HMF + Glucose  Sucrose     0.30
 H2O + Xylan -> Xylose             Xylan      90.00
 H2O + Xylan -> XyloseOligomer     Xylan       0.24
 Xylan -> 2 H2O + Furfural         Xylan       0.50
 Acetate -> AceticAcid             Acetate   100.00
 Lignin -> SolubleLignin           Lignin      0.50


Model the reaction:

In [8]:
feed = Stream(H2O=2.07e+05,
              Ethanol=18,
              H2SO4=1.84e+03,
              Sucrose=1.87,
              Extract=67.8,
              Acetate=25.1,
              Ash=4.11e+03,
              Lignin=1.31e+04,
              Protein=108,
              Glucan=180,
              Xylan=123,
              Arabinan=9.02,
              Mannan=3.08,
              Furfurals=172)
print('BEFORE REACTION')
feed.show()

# React feed molar flow rate
pretreatment_rxn(feed.mol)

print('AFTER REACTION')
feed.show()

BEFORE REACTION
Stream: d2
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): H2O        2.07e+05
                 Ethanol    18
                 H2SO4      1.84e+03
                 Sucrose    1.87
                 Extract    67.8
                 Acetate    25.1
                 Ash        4.11e+03
                 Lignin     1.31e+04
                 Protein    108
                 Glucan     180
                 Xylan      123
                 Arabinan   9.02
                 Mannan     3.08
                 Furfurals  172
AFTER REACTION
Stream: d2
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): H2O              2.07e+05
                 Ethanol          18
                 AceticAcid       25.1
                 Furfural         0.615
                 H2SO4            1.84e+03
                 HMF              0.546
                 Glucose          17.8
                 Xylose           111
                 Sucrose          1.86
                 Extract          67.8


### Reactions in series

SeriesReaction objects work the same way, but in series:

In [9]:
pretreatment_series_rxn = rxn.SeriesReaction(pretreatment_rxn)
pretreatment_series_rxn

SeriesReaction:
 stoichiometry                     reactant    X[%]
 H2O + Glucan -> Glucose           Glucan      9.90
 H2O + Glucan -> GlucoseOligomer   Glucan      0.30
 Glucan -> 2 H2O + HMF             Glucan      0.30
 Sucrose -> 2 H2O + HMF + Glucose  Sucrose     0.30
 H2O + Xylan -> Xylose             Xylan      90.00
 H2O + Xylan -> XyloseOligomer     Xylan       0.24
 Xylan -> 2 H2O + Furfural         Xylan       0.50
 Acetate -> AceticAcid             Acetate   100.00
 Lignin -> SolubleLignin           Lignin      0.50


In [10]:
import pandas as pd
print('NET CONVERSION IN PARALLEL')
print(pd.Series(pretreatment_rxn.X_net))

print('\nNET CONVERSION IN SERIES')
print(pd.Series(pretreatment_series_rxn.X_net))

NET CONVERSION IN PARALLEL
Glucan    0.105
Sucrose   0.003
Xylan     0.907
Acetate       1
Lignin    0.005
dtype: float64

NET CONVERSION IN Series
Glucan    8.91e-07
Sucrose      0.003
Xylan     1.08e-05
Acetate          1
Lignin       0.005
dtype: float64


In [11]:
feed = Stream(H2O=2.07e+05,
              Ethanol=18,
              H2SO4=1.84e+03,
              Sucrose=1.87,
              Extract=67.8,
              Acetate=25.1,
              Ash=4.11e+03,
              Lignin=1.31e+04,
              Protein=108,
              Glucan=180,
              Xylan=123,
              Arabinan=9.02,
              Mannan=3.08,
              Furfurals=172)
print('BEFORE REACTION')
feed.show()

# React feed molar flow rate
pretreatment_series_rxn(feed.mol)

print('AFTER REACTION')
feed.show()

BEFORE REACTION
Stream: d3
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): H2O        2.07e+05
                 Ethanol    18
                 H2SO4      1.84e+03
                 Sucrose    1.87
                 Extract    67.8
                 Acetate    25.1
                 Ash        4.11e+03
                 Lignin     1.31e+04
                 Protein    108
                 Glucan     180
                 Xylan      123
                 Arabinan   9.02
                 Mannan     3.08
                 Furfurals  172
AFTER REACTION
Stream: d3
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): H2O              2.07e+05
                 Ethanol          18
                 AceticAcid       25.1
                 Furfural         0.0614
                 H2SO4            1.84e+03
                 HMF              0.491
                 Glucose          17.8
                 Xylose           111
                 Sucrose          1.86
                 Extract          67.8

### Indexing reactions

Both SeriesReaction, and ParallelReaction objects are indexable:

In [12]:
# Index a slice
pretreatment_rxn[0:2]

ParallelReaction:
 stoichiometry                    reactant    X[%]
 H2O + Glucan -> Glucose          Glucan      9.90
 H2O + Glucan -> GlucoseOligomer  Glucan      0.30


In [13]:
# Index an item
pretreatment_rxn[0]

ReactionItem:
 stoichiometry            reactant    X[%]
 H2O + Glucan -> Glucose  Glucan      9.90


In [14]:
# Change conversion through the item
pretreatment_rxn[0].X = 0.10

In [15]:
pretreatment_rxn

ParallelReaction:
 stoichiometry                     reactant    X[%]
 H2O + Glucan -> Glucose           Glucan     10.00
 H2O + Glucan -> GlucoseOligomer   Glucan      0.30
 Glucan -> 2 H2O + HMF             Glucan      0.30
 Sucrose -> 2 H2O + HMF + Glucose  Sucrose     0.30
 H2O + Xylan -> Xylose             Xylan      90.00
 H2O + Xylan -> XyloseOligomer     Xylan       0.24
 Xylan -> 2 H2O + Furfural         Xylan       0.50
 Acetate -> AceticAcid             Acetate   100.00
 Lignin -> SolubleLignin           Lignin      0.50


Notice how changing conversion of a ReationItem object changes the converion in the ParallelReaction object.

### References

<a id='References'></a>

1. Humbird, D., Davis, R., Tao, L., Kinchin, C., Hsu, D., Aden, A., Dudgeon, D. (2011). Process Design and Economics for Biochemical Conversion of Lignocellulosic Biomass to Ethanol: Dilute-Acid Pretreatment and Enzymatic Hydrolysis of Corn Stover (No. NREL/TP-5100-47764, 1013269). https://doi.org/10.2172/1013269

2. Hatakeyama, T., Nakamura, K., & Hatakeyama, H. (1982). Studies on heat capacity of cellulose and lignin by differential scanning calorimetry. Polymer, 23(12), 1801–1804. https://doi.org/10.1016/0032-3861(82)90125-2

3. Thybring, E. E. (2014). Explaining the heat capacity of wood constituents by molecular vibrations. Journal of Materials Science, 49(3), 1317–1327. https://doi.org/10.1007/s10853-013-7815-6

4. Murphy W. K., and K. R. Masters. (1978). Gross heat of combustion of northern red oak (Quercus rubra) chemical components. Wood Sci. 10:139-141.